***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [38]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-9ed2  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [39]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [40]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  7 16:17 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [42]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [43]:
spark

In [44]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'yuval_206542839' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name.endswith(".parquet"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [45]:
"""
#only 1000 documents
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.limit(1000).select("title","id").rdd
"""

#full corpus 
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title","id").rdd


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [46]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [47]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [48]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

24/03/07 17:01:52 WARN SparkContext: The path /home/dataproc/inverted_index_gcp.py has been added already. Overwriting of added paths is not supported in the current version.


In [49]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [50]:
from nltk.stem import PorterStemmer

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

porter = PorterStemmer()

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]
    
  # Apply stemming to the tokens
  stemmed_tokens = [porter.stem(token) for token in tokens]
    
  # Count the frequency of each word using `Counter` from the collections module
  freq = Counter(stemmed_tokens)

  # Convert the `freq` dictionary to a list of tuples as required
  result = [(token, (id, count)) for token, count in freq.items()]

  return result



def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''

  # Sort the unsorted posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl



def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))



def partition_postings_and_write(postings, bucket_name):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning is done using the `token2bucket_id` function.

  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    bucket_name: str
      The name of the bucket where the posting lists will be stored.

  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket.
  '''
  # Map postings to buckets based on token
  postings_bucketed = postings.map(lambda posting: (token2bucket_id(posting[0]), posting))

  # Group postings by bucket ID
  postings_grouped = postings_bucketed.groupByKey().mapValues(list)

  # Write postings to disk and collect location information
  def write_postings_to_disk(postings_list):
      # Pass `bucket_name` to `write_a_posting_list` method
      return InvertedIndex.write_a_posting_list(postings_list, "postings_gcp/", bucket_name)

  # Apply the function to each group of postings and collect the results
  posting_locations = postings_grouped.map(write_postings_to_disk)

  return posting_locations

from math import log

def calculate_idf2(postings, total_documents):
  ''' Takes a posting list RDD, calculates the IDF for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
    total_documents: int
      Total number of documents in the corpus.

  Returns:
  --------
    RDD
      An RDD where each element is a (token, idf) pair.
  '''
  # Calculate Document Frequency (DF) for each token
  df_rdd = calculate_df(postings)

  # Calculate Inverse Document Frequency (IDF) for each token
  idf_rdd = df_rdd.map(lambda x: (x[0], log(total_documents / x[1])))

  return idf_rdd



In [51]:
#PROJECT 
"""
doc_text_pairs_titles = parquetFile.limit(1000).select("title","id").rdd

word_counts_title = doc_text_pairs_titles.flatMap(lambda x: word_count(x[0], x[1]))  #(token, (doc_id, tf))
postings_title = word_counts_title.groupByKey().mapValues(reduce_word_counts)
postings_title = postings_title.filter(lambda x: len(x[1])>10)

#create df dict
title2df = calculate_df(postings_title)
"""
FILTER_MIN = 10

# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_filtered = word_counts.groupByKey().mapValues(reduce_word_counts)

# filtering postings and calculate df
#postings_filtered = postings.filter(lambda x: len(x[1])>FILTER_MIN)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()




In [52]:
NUM_DOCUMENTS = 6348910

#create idf
w2idf_title = calculate_idf2(postings_filtered,NUM_DOCUMENTS)
w2idf_dict = w2idf_title.collectAsMap()

In [53]:
import math

#Multiply by number the tf-idf
MULT = 2**10

# Join the RDDs on the term
joined_rdd = postings_filtered.join(w2idf_title)

# Map over the joined RDD to multiply tf by idf for each document in the postings list MULT and floor
postings_title_tf_idf = joined_rdd.map(lambda x: (x[0], [(doc_id,math.floor(tf * x[1][1]*MULT)) for doc_id, tf in x[1][0]]))

In [54]:
postings_title_tf_idf.take(2)

[('tianjin',
  [(50821, 10588),
   (549078, 10588),
   (654214, 10588),
   (658408, 10588),
   (661627, 10588),
   (919903, 10588),
   (919904, 10588),
   (919906, 10588),
   (919907, 10588),
   (919910, 10588),
   (919912, 10588),
   (919913, 10588),
   (919914, 10588),
   (919915, 10588),
   (919916, 10588),
   (919917, 10588),
   (919920, 10588),
   (919921, 10588),
   (919922, 10588),
   (1004490, 10588),
   (1535845, 10588),
   (1557333, 10588),
   (1695424, 10588),
   (2173296, 10588),
   (2198075, 10588),
   (2198098, 10588),
   (2231881, 10588),
   (2554164, 10588),
   (2739178, 10588),
   (2998675, 10588),
   (3154771, 10588),
   (3221455, 10588),
   (3923205, 10588),
   (4074730, 10588),
   (4230794, 10588),
   (5051878, 10588),
   (5395184, 10588),
   (6389643, 10588),
   (6709954, 10588),
   (8102027, 10588),
   (8103431, 10588),
   (8554884, 10588),
   (8750443, 10588),
   (8838511, 10588),
   (9912931, 10588),
   (10584153, 10588),
   (10850810, 10588),
   (11096125, 1058

In [55]:
_ = partition_postings_and_write(postings_title_tf_idf, "yuval_206542839").collect()

In [56]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

In [57]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
inverted.idf = w2idf_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
- [1 files][ 94.9 MiB/ 94.9 MiB]                                                
Operation completed over 1 objects/94.9 MiB.                                     


In [58]:
!gsutil ls -lh $index_dst

 94.88 MiB  2024-03-07T17:23:47Z  gs://yuval_206542839/postings_gcp/index.pkl
TOTAL: 1 objects, 99491797 bytes (94.88 MiB)


In [59]:
!pip install google-cloud-storage
!pip install flask

In [60]:
from google.cloud import storage
from flask import Flask, request, jsonify
from pathlib import Path
import pyspark
from heapq import heappop, heappush, heapify
import numpy as np
from inverted_index_gcp import *
import pickle
from collections import Counter
import nltk
import math
from threading import Thread
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter, OrderedDict
from nltk.stem.porter import *
import re


bucket_name = "yuval_206542839"
index_src = "index"


# import inverted index from the bucket
base_dir = "postings_gcp/"
index_title = InvertedIndex.read_index(base_dir, f'{index_src}', bucket_name)
# bucket = get_bucket(bucket_name)
# blob = bucket.blob(file_path)
# contents = blob.download_as_bytes()
# index_title = pickle.loads(contents)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
query = "hello world"
tokens = preprocess_query(query)
tokens

['hello', 'world']

In [77]:
# w = "hello"
# index_title.read_a_posting_list("",w,"yuval_206542839")

In [78]:
query = " hello, running 12"
tokens = preprocess_query(query)
q = query_to_vector(tokens,index_title)
q

{'hello': 1, 'run': 1}

In [79]:
# cosine_similarity(q,index_title)

SEARCH ENGINE

In [105]:
from collections import Counter
import math
from collections import Counter
import math
from nltk.stem import PorterStemmer
import requests


def preprocess_query(query):
    """
    Preprocesses the query by removing stopwords, stemming, and tokenization.
    """
    
    # Stopwords list
    stop_words = set(stopwords.words('english')) 
    
    # Stemmer
    stemmer = PorterStemmer() 
    
    # lower case
    query = query.lower() 
    
    # Remove non-alphabetic characters
    query = re.sub(r'[^a-zA-Z\s]', '', query) 
   
    # Tokenize
    tokens = query.split() 

    # Remove stopwords and stem
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    
    return tokens


def query_to_vector(query_tokens, index):
    """
    Transforms a query into a vector using TF-IDF weights from the inverted index.
    :param query_tokens: List of tokens in the query
    :param index: The inverted index object
    :return: A dictionary representing the query vector, with terms as keys and TF-IDF as values.
    """
    # Calculate TF for the query
    query_tf = Counter(query_tokens)
    query_vector = {}

    for term, tf in query_tf.items():
        # Calculate TF-IDF, assuming the document contains the term
        if term in index.df:
            query_vector[term] = tf

    return query_vector


def cosine_similarity(query_vector,index):
    """
    Calculates the cosine similarity between a query vector and document vectors,
    where document vectors are represented by posting lists of tokens that appear both in the documents and the query.
    :param query_vector: The query vector represented as a dictionary.
    :param posting_lists: A dictionary of posting lists, with terms as keys and lists of (doc_id, tf_idf) as values.
    :return: A list of tuples, each tuple being (doc_id, cosine_similarity_score).
    """
    # Initialize document scores and norms
    doc_scores = Counter()
    doc_norms = Counter()

    # Calculate scores for each term in the query
    for term, query_weight in query_vector.items():
        if term in index.df:
            posting = index.read_a_posting_list("",term,"yuval_206542839")
            for doc_id, tf_idf in posting:
                doc_scores[doc_id] += query_weight * tf_idf
                doc_norms[doc_id] += tf_idf ** 2

    # Calculate query norm
    sum_total = 0
    for weight in query_vector.values():
        sum_total += weight**2
    query_norm = math.sqrt(sum_total)

    # Calculate final cosine similarity scores
    similarities = []
    for doc_id, score in doc_scores.items():
        doc_norm = math.sqrt(doc_norms[doc_id])
        if query_norm == 0 or doc_norm == 0:
            similarity = 0
        else:
            similarity = score / (query_norm * doc_norm)
        similarities.append((doc_id, similarity))

    return similarities[:100]


def return_result(page_ids):
    # recieve list of page_idf
    # return list (doc_id,title)
    titles = []
    for page_id in page_ids:
        url = f'https://en.wikipedia.org/?curid={page_id}'
        response = requests.get(url)
        if response.status_code == 200:
            # Assuming the title is easily extractable from the response content
            # This is a simplification; actual extraction may require parsing the HTML or using an API
            title_start = response.text.find('<title>') + 7
            title_end = response.text.find('</title>', title_start)
            title = response.text[title_start:title_end].replace(" - Wikipedia", "")
            titles.append((page_id,title))
            
    return titles
            



def my_search(query):
    """
    Performs the search operation, including query preprocessing, document retrieval,
    and document ranking.
    """
    index = index_title
    tokens = preprocess_query(query)
    query_vector = query_to_vector(tokens, index)
    sim = cosine_similarity(query_vector,index)
    page_ids = [doc_id for doc_id, _ in sim]
    result = return_result(page_ids)
    
    return result



In [106]:
q = "hello, running"
my_search(q)

[(13834, '"Hello, World!" program'),
 (54295, 'Hello Kitty'),
 (99032, 'Hello, Dolly! (musical)'),
 (244825, 'Hello I Must Be Going'),
 (479467, 'Hello, I Must Be Going (book)'),
 (479470, 'Hello, I Must Be Going! (album)'),
 (576891, 'Hello Cheeky'),
 (619375, 'Hello (The Beloved song)'),
 (710794, 'Hello Nasty'),
 (899002, 'Hello Sailor (band)'),
 (997206, 'Hello Brother (1999 film)'),
 (1122016, 'Hello! (magazine)'),
 (1206479, 'Hello, Goodbye'),
 (1231628, 'Hello (Poe album)'),
 (1241470, 'Hello! Project'),
 (1379058, 'Hello Muddah, Hello Fadduh (A Letter from Camp)'),
 (1480256, 'Hello! Morning'),
 (1519830, 'Hello! Project Shuffle Unit'),
 (1597633, 'Ernest Hello'),
 (1790190, 'Hello Gutter, Hello Fadder'),
 (1879210, 'Say Hello to Sunshine'),
 (1895130, 'Hello, Dolly!'),
 (1895881, 'Hello, Dolly! (song)'),
 (1901017, 'Hello, Dolly! (Ella Fitzgerald album)'),
 (1929746, 'Hello (airline)'),
 (1977205, 'World Hello Day'),
 (2180150, 'Hello Lisa'),
 (2213578, 'Hello, Mannequin'),
 (

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [92]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''

  edges = pages.flatMap(lambda page: [(page[0], linked_page[0]) for linked_page in page[1]]).distinct()

  # Extract vertices from edges
  # Vertices are unique IDs from both sources and destinations
  vertices = edges.flatMap(lambda edge: [edge[0], edge[1]]).distinct().map(lambda id: (id,id))

  return edges, vertices


In [ ]:
t_start = time()
# pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 2 hours
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = "21.39$" 
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag